# Import Data

In [ ]:
! unzip -q images-train.zip
! unzip -q images-val.zip
! unzip -q images-test.zip

In [ ]:
!pip install -q timm
!pip install -q transformers

     |████████████████████████████████| 549 kB 26.6 MB/s 
     |████████████████████████████████| 182 kB 76.0 MB/s 
     |████████████████████████████████| 5.8 MB 26.6 MB/s 
     |████████████████████████████████| 7.6 MB 50.0 MB/s 


In [ ]:
import shutil
import torch
from torch.utils.data import DataLoader,Dataset
import transformers
from transformers import CLIPModel, CLIPConfig
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import cv2 as cv
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import WeightedRandomSampler
from sklearn.metrics import  f1_score
from tqdm.autonotebook import tqdm
import json
from copy import deepcopy
from getpass import getpass
import urllib


from transformers import BertTokenizer, VisualBertForPreTraining, VisualBertModel, BertModel
from transformers import ViTFeatureExtractor, ViTModel

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile


from tensorboard.plugins import projector

In [ ]:
len(os.listdir('images-train')), len(os.listdir('images-val'))

(3, 2)

In [ ]:
category_dict = {
    'Support_Multimodal' : 0,
    'Support_Text' : 1,
    'Insufficient_Multimodal' : 2,
    'Insufficient_Text': 3,
    'Refute': 4
}

In [ ]:
train_df = pd.read_csv('train.csv',sep='\t')
train_df['Category'] = train_df['Category'].replace(to_replace=category_dict)
train_df.head()

,Unnamed: 0,claim,claim_image,document,document_image,Category,Claim OCR,Document OCR
0,0,Delhi: Seventh round of meeting between Centra...,http://pbs.twimg.com/media/Eq4GUZmVoAAiSz4.jpg...,In their seventh attempt to end the logjam ove...,http://pbs.twimg.com/media/Eq4I0YaUcAcXQnO.jpg...,0,ANI,संजय अग्रवाल\nपीयूष गोयल\nमे\nद्र\nd\nANI
1,1,"Through a massive effort around 22,000 Tableeg...",http://pbs.twimg.com/media/EUwQLlEUcAA_JoY.jpg...,"\nSep 08, 2021-Wednesday\n \n\nMetro cities - ...",http://pbs.twimg.com/media/EVURc_nUUAA_P7O.jpg...,2,+\nANI\nA\nHOT\nBo\nसत्यमेव जयते\nINDI,LEUR\nANI\nJWA\nORD\nसत्यमेव जयते\nINDI
2,2,"Hubli, Karnataka: Prime Minister Narendra Modi...",http://pbs.twimg.com/media/DzDGuEkU8AAsNYP.jpg...,Prime Minister Narendra Modi addressed rallies...,http://pbs.twimg.com/media/DzDHPbyV4AAarEF.jpg...,1,"Serday 16th February, 2019\nDharwad, Karnataks...",ಇಂಧನ ಸುರ\nat\nರಾಷ್ಟ್ರಕ್ಕೆ ಸಮರ್ಪಣೆ\nನಗರ ಅ\nಬರ್ತ...
3,3,Video showing Chinese soldier captured by Indi...,https://www.boomlive.in/h-upload/2020/06/18/92...,An undated video showing Indo Tibetan Border P...,https://www.boomlive.in/h-upload/2020/06/18/92...,4,Forwarded\nभारतीय सेना ने आखिर कर ही दी चीन की...,Chinese vehicle and a soldier captured in Lada...
4,4,Image shows Jagdeep Dhankhar attending RSS eve...,https://www.boomlive.in/h-upload/2020/05/03/92...,A morphed image of West Bengal governor Jagdee...,https://www.boomlive.in/h-upload/2020/05/03/92...,4,Mamata Banerjee Supporters (FAM)\n@FAM4TMC\n.@...,Mamata Banerjee Supporters (FAM)\nOFAMITMC\nFo...


In [ ]:
val_df = pd.read_csv('val.csv',sep='\t')
val_df['Category'] = val_df['Category'].replace(to_replace=category_dict)
val_df.head()

,Unnamed: 0,claim,claim_image,document,document_image,Category,Claim OCR,Document OCR
0,0,Musician Kodak Black was shot outside of a nig...,https://www.digitalmusicnews.com/wp-content/up...,"On 26 December 2016, the web site Gummy Post p...",https://www.snopes.com/tachyon/2017/01/caution...,4,,CAUTION CAUTION CAUTION
1,1,Farmers' #RepublicDay tractor rally in protest...,http://pbs.twimg.com/media/EsoL4w7VEAAw-YR.jpg...,Thousands of protesting farmers clashed with p...,http://pbs.twimg.com/media/EsoLcWvUwAA0EWd.jpg...,0,SA\nANI,ANI
2,2,Uttarakhand Chief Minister Trivendra Singh Raw...,http://pbs.twimg.com/media/D-t-R6DXoAAE8xX.jpg...,Union Budget 2019 : Finance Minister Nirmala S...,http://pbs.twimg.com/media/D-sq_M9U8AAN2q9.png...,1,PHILIP\nANI\n|ANI\nMaking Uttarakhand Digit\nITDA,NaN
3,3,Amazon says the coronavirus outbreak has cause...,https://cdn.cnn.com/cnnnext/dam/assets/2003131...,"Amazon says it needs to hire 100,000 people ac...",https://s.abcnews.com/images/US/WireAP_b1f4f77...,0,Exfoliates Skin\nPamper Yourself\nSKLY\nHongel...,0:0\n44\nchute\n101\nOV
4,4,Freshman Republican Sen. Martha McSally of Ari...,http://pbs.twimg.com/media/D1ADmYzX4AA0bkw.jpg...,Martha McSally was the the first female fighte...,http://pbs.twimg.com/media/D1ADT33UwAAZ2_-.jpg...,0,NaN,MS. McSALLY\nabc\nNEWS


In [ ]:
test_df = pd.read_csv('test.csv',sep='\t')
test_df['Category'] = test_df['Category'].replace(to_replace=category_dict)
test_df.head()

,Unnamed: 0,claim,claim_image,document,document_image,Category,Claim OCR,Document OCR
0,0,Delhi: Group of students protest outside offic...,http://pbs.twimg.com/media/EOJMHR4VUAALSeG.jpg...,Hundreds of Jamia Millia Islamia students prot...,http://pbs.twimg.com/media/EOJ4Q47UcAAq6wy.jpg...,2,ANI,NaN
1,1,"SpaceX's Crew Dragon capsule, its first spacec...",http://pbs.twimg.com/media/D0rEcWcX0AAe1x8.png...,The unmanned craft took about 27 hours to reac...,http://pbs.twimg.com/amplify_video_thumb/11017...,2,TONY GRAY NASA,abc\nNEWS
2,2,Gujarat: Prime Minister Narendra Modi enroute ...,http://pbs.twimg.com/media/ERhiInXVAAAv8vp.jpg...,"\nSep 07, 2021-Tuesday\n \n\nMetro cities - D...",http://pbs.twimg.com/media/ERhk0YKX0AEOlrT.jpg...,2,ANI,RANGE\nees\n1111\nROVER\n2CA3652\nANI
3,3,Maharashtra: A trainee aircraft of Carver avia...,http://pbs.twimg.com/media/DyoKIfzUUAAV6pI.jpg...,A trainer aircraft crashed in an empty plot in...,http://pbs.twimg.com/media/DyqWf_hVAAAI13H.jpg...,3,AR\nVT-RDX\nANI,VT-ROX
4,4,An association of pediatricians “admitted” tha...,https://www.snopes.com/tachyon/2016/01/HPV-vac...,"On 28 January 2016, the web site Truth Kings p...",https://www.snopes.com/tachyon/2016/01/HPV-vac...,4,Good\nPediatricians Association Admits HPV Vac...,Good\nPediatricians Association Admits HPV Vac...


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.7 MB/s 
     |████████████████████████████████| 1.3 MB 68.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=bf1fd108d683f0c06a2ed9cdc67faa903aae89568f9a025c0595879e04359f28
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
import os
for i in ['Insufficient_Multimodal','Insufficient_Text','Support_Multimodal','Support_Text','Refute']:
  fls = os.listdir('images-train/claim/'+i+'/')
  for f in fls:
      os.rename('images-train/claim/'+i+'/'+f, 'images-train/claim/'+f)
  os.rmdir('images-train/claim/'+i+'/')
  fls = os.listdir('images-train/document/'+i+'/')
  for f in fls:
      os.rename('images-train/document/'+i+'/'+f, 'images-train/document/'+f)
  os.rmdir('images-train/document/'+i+'/')

for i in ['Insufficient_Multimodal','Insufficient_Text','Support_Multimodal','Support_Text','Refute']:
  fls = os.listdir('images-val/claim/'+i+'/')
  for f in fls:
      os.rename('images-val/claim/'+i+'/'+f, 'images-val/claim/'+f)
  os.rmdir('images-val/claim/'+i+'/')
  fls = os.listdir('images-val/document/'+i+'/')
  for f in fls:
      os.rename('images-val/document/'+i+'/'+f, 'images-val/document/'+f)
  os.rmdir('images-val/document/'+i+'/')

In [ ]:
'''
  Dataset class with feature extractor
'''
class FactifyData(Dataset):


    def __init__(self,root_dir,claim_img,doc_img,claim_text,doc_text,category,feature_extractor, sbert_model, transforms=None):

        self.claim_img = claim_img
        self.doc_img = doc_img
        self.claim_text = claim_text
        self.doc_text = doc_text
        self.root_dir = root_dir
        self.feature_extractor = feature_extractor
        self.sbert_model = sbert_model
        self.category = category

    def __len__(self):
        return len(self.claim_img)


    def __getitem__(self,idx):
        claim_img = cv.imread(self.root_dir+"/claim/"+str(self.claim_img[idx])+".jpg")
        claim_img = self.feature_extractor(claim_img, return_tensors="pt")['pixel_values'][0]
        doc_img = cv.imread(self.root_dir+"/document/"+str(self.doc_img[idx])+".jpg")
        doc_img = self.feature_extractor(doc_img, return_tensors="pt")['pixel_values'][0]
        claim_text = self.sbert_model.encode(self.claim_text[idx])
        doc_text = self.sbert_model.encode(self.doc_text[idx])
        return {
            'claim_img': claim_img,
            'claim_text': claim_text,
            'doc_img': doc_img,
            'doc_text': doc_text,
            'category': torch.tensor(self.category[idx],dtype=torch.long),
        }



def create_dataset(df,feature_extractor, sbert_model, folder):
    ds = FactifyData(
        root_dir = folder,
        claim_img = df['Unnamed: 0'].to_list(),
        doc_img = df['Unnamed: 0'].to_list(),
        claim_text = df['claim'].to_list(),
        doc_text = df['document'].to_list(),
        category = df['Category'].to_list(),
        feature_extractor=feature_extractor,
        sbert_model = sbert_model,
        transforms= torchvision.transforms.Compose(
            [
              torchvision.transforms.ToTensor()
            ]
        )
    )
    return ds

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
sbert_model = SentenceTransformer('stsb-mpnet-base-v2')

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/868 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.67k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/588 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
# Create dataset
train_dataset = create_dataset(train_df, feature_extractor, sbert_model,  'images-train')
test_dataset = create_dataset(test_df, feature_extractor, sbert_model, 'images-test')
val_dataset = create_dataset(val_df,feature_extractor, sbert_model, 'images-val')

In [ ]:
'''
  Loaders for training
'''
BATCH_SIZE = 16

train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset,batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset,batch_size=BATCH_SIZE)

train_loaders = []
for i in range(7):
  df_temp = train_df.iloc[5000*i:5000*(i+1),:]
  train_dataset = create_dataset(df_temp, feature_extractor, sbert_model,  'images-train')
  train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE)
  train_loaders.append(train_loader)

In [ ]:
class BertViT(torch.nn.Module):

    def __init__(self, device, num_classes=3 ):
        super(BertViT, self).__init__()
        self.device = device

        # self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        # self.bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
        # self.resnet = torchvision.models.resnet50(pretrained=True).to(device)
        self.vit = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)


        self.num_classes = num_classes
        self.fc1 = nn.Linear(3072, 512).to(device)
        self.relu = nn.ReLU().to(device)
        self.batchnorm = nn.BatchNorm1d(512).to(device)
        self.fc2 = nn.Linear(512, self.num_classes).to(device)
        self.drop = nn.Dropout(0.5)


        # for params in self.bert_model.parameters():
        #   params.requires_grad = False


        for params in self.vit.parameters():
          params.requires_grad = False



    def forward(self, claim_img_list, doc_img_list, claim_text, doc_text ):


        # bert_out = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)

        # print(bert_out.pooler_output.shape)
        claim_img_list = {'pixel_values': claim_img_list}
        doc_img_list = {'pixel_values': doc_img_list}
        claim_img_outputs = self.vit(**claim_img_list).pooler_output
        doc_img_outputs = self.vit(**doc_img_list).pooler_output

        out = torch.cat((claim_img_outputs, claim_text, doc_img_outputs, doc_text ), dim=1)

        out =  self.fc1(out)
        out = self.batchnorm(out)
        out = self.relu(out)
        out = self.drop(out)
        out = self.fc2(out)

        return out



In [ ]:
vis_model = BertViT(device, 5)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vis_model.parameters(), lr=3e-4, betas=(0.5,0.99))

In [ ]:
for epoch in range(7):

    print('Epoch ',epoch+1)

    running_loss = 0.0

    correct = 0
    total = 0
    training_loss = []
    y_true = []
    y_predicted = []
    train_loader = train_loaders[epoch]
    for i, x in enumerate(train_loader):
        # print(x['img'].shape)

        optimizer.zero_grad()
        outputs = vis_model(x['claim_img'].to(device),x['doc_img'].to(device),x['claim_text'].to(device),x['doc_text'].to(device))

        _, predicted = torch.max(outputs.data, 1)
        total += x['category'].to(device).size(0)
        y_true.append(x['category'].to(device))
        y_predicted.append(predicted)
        correct += (predicted.cpu() == x['category']).sum().item()

        loss = criterion(outputs, x['category'].to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        training_loss.append(loss.item())
        if (i+1) % 50 == 0:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

    y_true = torch.cat(y_true,dim=0)
    y_predicted = torch.cat(y_predicted,dim=0)

    train_f1_s = f1_score(y_true.cpu().numpy(),y_predicted.cpu().numpy(),average='weighted')

    print(f'Train Acc : {100*correct/total}')
    print(f'Train F1 : {train_f1_s}')
    print(f'Val Acc : ',end=' ')

    correct = 0
    total = 0
    y_true = []
    y_predicted = []

    with torch.no_grad():
        for x in val_loader:

            labels = x['category'].to(device)

            outputs = vis_model(x['claim_img'].to(device),x['doc_img'].to(device),x['claim_text'].to(device),x['doc_text'].to(device))

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            y_true.append(x['category'].to(device))
            y_predicted.append(predicted)
            correct += (predicted == labels).sum().item()

    y_true = torch.cat(y_true,dim=0)
    y_predicted = torch.cat(y_predicted,dim=0)
    val_f1_s = f1_score(y_true.cpu().numpy(),y_predicted.cpu().numpy(),average='weighted')
    print(100 * correct / total)
    print('Val Macro F1',val_f1_s)

print('Finished Training')


In [ ]:
correct = 0
total = 0
y_true = []
y_predicted = []
predicted_labels = []

with torch.no_grad():
    for x in test_loader:
        labels = x['category'].to(device)

        outputs = vis_model(x['claim_img'].to(device),x['doc_img'].to(device),x['claim_text'].to(device),x['doc_text'].to(device))

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        y_true.append(x['category'].to(device))
        y_predicted.append(predicted)
        correct += (predicted == labels).sum().item()

y_true = torch.cat(y_true,dim=0)
y_predicted = torch.cat(y_predicted,dim=0)
val_f1_s = f1_score(y_true.cpu().numpy(),y_predicted.cpu().numpy(),average='weighted')
print(100 * correct / total)
print('Test Macro F1',val_f1_s)
print(y_true)
print(y_predicted)

65.94666666666667
Test Macro F1 0.6499773114051051
tensor([2, 2, 2,  ..., 3, 1, 4], device='cuda:0')
tensor([0, 0, 0,  ..., 3, 3, 4], device='cuda:0')


In [ ]:
l=y_predicted.cpu().numpy()

In [ ]:
rev_category_dict = {
    0: 'Support_Multimodal',
    1: 'Support_Text',
    2: 'Insufficient_Multimodal',
    3: 'Insufficient_Text',
    4: 'Refute'
}

In [ ]:
ll=[]
for i in l:
  ll.append(rev_category_dict[i])
print(ll[-4:])
print(l[-4:])

['Support_Multimodal', 'Insufficient_Text', 'Insufficient_Text', 'Refute']
[0 3 3 4]
